In [1]:
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [2]:
from zipfile import ZipFile

# 훈련 이미지 데이터 압축 풀기
with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
    
# 테스트 이미지 데이터 압축 풀기
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

In [3]:
import torch # 파이토치 
import random
import numpy as np

# 시드 값 고정
def set_seed(seed):
#     torch.use_deterministic_algorithms(True)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(10)

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
device

device(type='cuda')

In [6]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(labels, 
                                test_size=0.1,
                                stratify=labels['has_cactus'],
                                random_state=10)

In [7]:
print('훈련 데이터 개수:', len(train))
print('검증 데이터 개수:', len(valid))

훈련 데이터 개수: 15750
검증 데이터 개수: 1750


In [8]:
import cv2 # OpenCV 라이브러리
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
    # 초기화 메서드(생성자) ---①
    def __init__(self, df, img_dir = './', transform=None):
        super().__init__() # 상속받은 Dataset의 __init__() 메서드 호출 ---②
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # 데이터 세트 크기 반환 메서드 ---③
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드 ---④
    def __getitem__(self, idx):
        img_path = self.img_dir + self.df.iloc[idx, 0] # 이미지 파일 경로 
        image = cv2.imread(img_path) # 이미지 파일 읽기 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        label = self.df.iloc[idx, 1] # 이미지 레이블(타깃 값)
        # 이미지 변환
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [9]:
from torchvision import transforms # 이미지 변환을 위한 모듈

transform = transforms.ToTensor()

In [10]:
dataset_train = ImageDataset(df=train, img_dir='train/', transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform)

In [11]:
from torch.utils.data import DataLoader # 데이터 로더 생성을 위한 클래스

loader_train = DataLoader(dataset=dataset_train, batch_size=64, 
                          shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=64, 
                          shuffle=False)

In [12]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

class Model(nn.Module):
    # 신경망 계층 정의 ---①
    def __init__(self):
        super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출
        # 첫 번째 합성곱 계층 ---②
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, 
                               kernel_size=3, padding=2) 
        # 두 번째 합성곱 계층 ---③
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, 
                               kernel_size=3, padding=2) 
        # 최대 풀링 계층 ---④
        self.max_pool = nn.MaxPool2d(kernel_size=2) 
        # 평균 풀링 계층 ---⑤
        self.avg_pool = nn.AvgPool2d(kernel_size=2) 
        # 전결합 계층 ---⑥
        self.fc = nn.Linear(in_features=64 * 4 * 4, out_features=2)
        
    # 순전파 출력 정의 ---⑦
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 4 * 4) # 평탄화
        x = self.fc(x)
        return x

In [13]:
# class Model(nn.Module):
#     def __init__(self):
#         super().__init__() 
#         # 첫 번째 합성곱, 최대 풀링 계층
#         self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, 
#                                               out_channels=32, 
#                                               kernel_size=3, 
#                                               padding=2),
#                                     nn.ReLU(),
#                                     nn.MaxPool2d(kernel_size=2))
#         # 두 번째 합성곱, 최대 풀링 계층
#         self.layer2 = nn.Sequential(nn.Conv2d(in_channels=32, 
#                                               out_channels=64, 
#                                               kernel_size=3, 
#                                               padding=2),
#                                     nn.ReLU(),
#                                     nn.MaxPool2d(kernel_size=2))
#         # 평균 풀링 계층
#         self.avg_pool = nn.AvgPool2d(kernel_size=2) 
#         # 전결합 계층
#         self.fc = nn.Linear(in_features=64 * 4 * 4, out_features=2)
        
#     # 순전파 출력 정의
#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.avg_pool(x)
#         x = x.view(-1, 64 * 4 * 4) # 평탄화
#         x = self.fc(x)
#         return x

In [14]:
model = Model().to(device)

model

Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)

In [15]:
# 손실함수
criterion = nn.CrossEntropyLoss()

In [16]:
# 옵티마이저
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [17]:
import math

math.ceil(len(train) / 64)

247

In [18]:
len(loader_train)

247

In [19]:
epochs = 10 # 총 에폭

# 총 10에폭만큼 훈련 ---①
for epoch in range(epochs):
    epoch_loss = 0 # 에폭별 손실값 초기화
    # 미니배치 크기만큼 데이터를 추출하는 작업을 '반복 횟수'만큼 반복 ---②
    for images, labels in loader_train:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 ---③
        images = images.to(device)
        labels = labels.to(device)

        # 옵티마이저 내 기울기 초기화 ---④
        optimizer.zero_grad()
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산 ---⑤
        outputs = model(images)
        # 손실함수를 활용해 outputs와 labels의 손실값 계산 ---⑥
        loss = criterion(outputs, labels)
        # 역전파 수행 ---⑦
        loss.backward()
        # 가중치 갱신 ---⑧
        optimizer.step()
        
        epoch_loss += loss.item() # 현재 배치에서의 손실 추가
        
    print(f'에폭: [{epoch+1}/{epochs}], 손실값: {epoch_loss/len(loader_train):.4f}')

에폭: [1/10], 손실값: 0.5462
에폭: [2/10], 손실값: 0.4818
에폭: [3/10], 손실값: 0.3522
에폭: [4/10], 손실값: 0.2784
에폭: [5/10], 손실값: 0.2467
에폭: [6/10], 손실값: 0.2138
에폭: [7/10], 손실값: 0.1910
에폭: [8/10], 손실값: 0.1827
에폭: [9/10], 손실값: 0.1729
에폭: [10/10], 손실값: 0.1583


In [20]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 예측 확률값과 실제값을 담을 리스트 초기화
true_list = []
preds_list = []

model.eval() # 모델을 평가 상태로 설정 ---①

with torch.no_grad(): # 기울기 계산 비활성 ---②
    for images, labels in loader_valid:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1] # 예측 확률값 ---③
        true = labels.cpu() # 실제값 ---④
        # 예측 확률값과 실제값을 리스트에 추가
        preds_list.extend(preds)
        true_list.extend(true)
        
    # 검증 데이터 ROC AUC 점수 계산 ---⑤
    print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')

검증 데이터 ROC AUC : 0.9867


In [21]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform=transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=64, 
                         shuffle=False)

In [22]:
model.eval() # 모델을 평가 상태로 설정

preds = [] # 타깃 예측 값 저장용 변수 초기화 ---①

with torch.no_grad(): # 기울기 계산 비활성
    for images, _ in loader_test:
        # 이미지 데이터 미니배치를 장비에 할당
        images = images.to(device)
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 타깃 값이 1일 확률(예측 값) ---②
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        # preds에 preds_part 이어붙이기 ---③
        preds.extend(preds_part)

In [23]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)

In [24]:
submission

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,0.982228
1,0017242f54ececa4512b4d7937d1e21e.jpg,0.999730
2,001ee6d8564003107853118ab87df407.jpg,0.113825
3,002e175c3c1e060769475f52182583d0.jpg,0.364750
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.949532
...,...,...
3995,ffaafd0c9f2f0e73172848463bc2e523.jpg,0.989928
3996,ffae37344310a1549162493237d25d3f.jpg,0.999928
3997,ffbd469c56873d064326204aac546e0d.jpg,0.998974
3998,ffcb76b7d47f29ece11c751e5f763f52.jpg,0.990885


In [25]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')